# NOTES:

- This is like what Cadence was wroking on, but after normalization etc and without correlation to behavior (and do sub-group analysis, logistic regression). So these need to be added.

# Import stuff

In [1]:
import os
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import pandas as pd

# Params

In [2]:
#dirs:
rs_data_path_IDCH_sub_Nums_normed_concat = 'data/pre_proc_data_IDCH_subNums_normed_concat' # Here each voxel time series will be normed and concatenated
masks_dir = 'masks'
connectiviity_data_dir = 'data/connectivity_data'
time_series_dir = 'data/time_series'

# file_format:
any_session_file_format = '_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz'

# R01 to IDCH mapping:
mapping_R01_to_IDCH = {'222': '101', '183': '102', '216': '103', '192': '104', '251': '105', '206': '106', '180': '107', '184': '108', '169': '109', '207': '110',
                       '159': '111', '115': '112', '114': '113', '232': '114', '173': '115', '171': '117', '215': '118', '265': '119', '177': '120', '269': '121',
                       '261': '122'}

mapping_R01_to_IDCH_more = {'094': '123', '253': '124', '247': '125', '233': '126', '229': '127', '246': '128', '214': '129', '205': '130', '234': '131', '230': '132',
                                        '218': '133', '213': '134', '237': '135', '129': '136', '126': '137', '228': '138', '141': '139', '197': '140', '202': '141', '196': '142',
                                        '181': '143', '189': '144', '142': '145', '176': '148', '163': '149', '099': '152', '052': '153', '063': '154', '082': '155'}

# concatenate the two mappings:
mapping_R01_to_IDCH.update(mapping_R01_to_IDCH_more)
# masks: * mask files should end with _mask.nii.gz
masks_to_apply = ['L_SMA_mask.nii.gz', 'R_SMA_mask.nii.gz', 'L_post_putamen_mask.nii.gz', 'R_post_putamen_mask.nii.gz', 'R_premotor_mask.nii.gz', 'L_premotor_mask.nii.gz', 'L_anterior_caudate_mask.nii.gz', 'R_anterior_caudate_mask.nii.gz', 'vmpfc_mask.nii.gz', 'L_vlpfc_mask.nii.gz', 'R_vlpfc_mask.nii.gz', 'L_anterior_putamen_mask.nii.gz', 'R_anterior_putamen_mask.nii.gz', 'R_frontopolar_mask.nii.gz']

# regions to test connectivity between: * add pairs of regions to test connectivity between
connectivity_regions = [['L_SMA', 'L_post_putamen'], ['R_SMA', 'R_post_putamen'], ['L_premotor', 'L_post_putamen'], ['R_premotor', 'R_post_putamen'], ['L_anterior_caudate', 'vmpfc'], ['R_anterior_caudate', 'vmpfc'], ['L_vlpfc', 'L_post_putamen'], ['R_vlpfc', 'R_post_putamen'], ['L_vlpfc', 'L_anterior_putamen'], ['R_vlpfc', 'R_anterior_putamen'], ['L_vlpfc', 'L_anterior_caudate'], ['R_vlpfc', 'R_anterior_caudate'], ['R_frontopolar', 'R_anterior_caudate'], ['R_frontopolar', 'R_post_putamen'], ['R_post_putamen', 'L_post_putamen'], ['R_anterior_caudate', 'L_anterior_caudate']]


# Create IDCH sub id folders + Extract time series using masks (directly from the data) for each session separately

In [4]:
# get all directories in the root directory
subs_files = os.listdir(rs_data_path_IDCH_sub_Nums_normed_concat)
subs_files = [f for f in subs_files if f.endswith('.nii.gz')]
subs_files.sort()
subs_files

for sub_file in subs_files:    
    # get sub (IDCH) id:
    sub_id = sub_file.split('_')[0].split('-')[1]
    print(f'>> Processing sub-{sub_id}')

    # create a new foler for the sub according to sub_id:
    sub_ts_dir = os.path.join(time_series_dir, 'sub-' + sub_id)
    if not os.path.exists(sub_ts_dir):
        os.makedirs(sub_ts_dir)

    for mask in masks_to_apply:
        mask_name = mask.split('_mask')[0]
        input_file = os.path.join(rs_data_path_IDCH_sub_Nums_normed_concat, sub_file)
        output_file = os.path.join(sub_ts_dir, f"sub-{sub_id}_IDCH_{mask_name}_time_series.txt")
        if os.path.exists(output_file):
            continue
        print(f'>> Extracting time series for sub-{sub_id}, using {mask}:')
        # fslmeants -i <input_file> -o <output_file> -m <mask_file>
        os.system(f'fslmeants -i {input_file} -o {output_file} -m {os.path.join(masks_dir, mask)}')


>> Processing sub-101
>> Processing sub-102
>> Processing sub-103
>> Processing sub-104
>> Processing sub-105
>> Processing sub-106
>> Processing sub-107
>> Processing sub-108
>> Processing sub-109
>> Processing sub-110
>> Processing sub-111
>> Processing sub-112
>> Processing sub-113
>> Processing sub-114
>> Processing sub-115
>> Processing sub-117
>> Processing sub-118
>> Processing sub-119
>> Processing sub-120
>> Processing sub-121
>> Processing sub-122
>> Processing sub-123
>> Processing sub-124
>> Processing sub-125
>> Processing sub-126
>> Processing sub-127
>> Processing sub-128
>> Processing sub-129
>> Processing sub-130
>> Processing sub-131
>> Processing sub-132
>> Processing sub-133
>> Processing sub-134
>> Processing sub-135
>> Processing sub-136
>> Processing sub-137
>> Processing sub-138
>> Processing sub-139
>> Processing sub-140
>> Processing sub-141
>> Processing sub-142
>> Processing sub-143
>> Processing sub-144
>> Processing sub-145
>> Processing sub-148
>> Process

# Calculate correlations (connectivity) and plot each subject's timeseries data

In [5]:
# get sub dirs:
IDCH_sub_dirs = [x for x in os.listdir(time_series_dir) if 'sub-' in x]
# sort the sub dirs:
IDCH_sub_dirs.sort()

# List to collect data for DataFrame
per_region_data = {}

# Loop over the subject directories in the root directory
for region_pair in connectivity_regions:
    print(f"Analyzing connectivity between {region_pair[0]} and {region_pair[1]}")
    region_df = []
    for subj in IDCH_sub_dirs:
        subject_dir = os.path.join(time_series_dir, subj)
        sub_ID = subj.split('-')[1]
        print(f"Analyzing subject {sub_ID}")

        # Get the data for both regions:
        reg1_dir = os.path.join(subject_dir, f'sub-{sub_ID}_IDCH_{region_pair[0]}_time_series.txt')
        reg2_dir = os.path.join(subject_dir, f'sub-{sub_ID}_IDCH_{region_pair[1]}_time_series.txt')

        reg1_ts_data = np.loadtxt(reg1_dir)
        reg2_ts_data = np.loadtxt(reg2_dir)

        # Calculate the Pearson correlation between the time series
        correlation, p_val = pearsonr(reg1_ts_data, reg2_ts_data)
        z_score = 0.5 * np.log((1 + correlation) / (1 - correlation))

        # Print correlation for the subject:
        print(f">> Subject {sub_ID}: correlation between {region_pair[0]} and {region_pair[1]}: r={correlation}, p={p_val}, z-score={z_score}")

        # # Scatterplot
        # fig, axs = plt.subplots(1, 2, figsize=(12, 5), sharey=False)
        # axs[0].scatter(reg1_ts_data, reg2_ts_data)
        # axs[0].set_title(f'Scatterplot for Subject {sub_ID}')
        # axs[0].set_xlabel(region_pair[0])
        # axs[0].set_ylabel(region_pair[1])
        # # Line Plot for the Time-Series Data
        # axs[1].plot(reg1_ts_data, color='red', label=region_pair[0])
        # axs[1].plot(reg2_ts_data, color='blue', label=region_pair[1])
        # axs[1].set_title(f'Time Series for Subject {sub_ID}')
        # axs[1].set_xlabel('Time')
        # axs[1].set_ylabel('BOLD Signal')
        # axs[1].legend()
        # plt.show()

        #Append the data to list (with means)
        region_df.append({"subID": sub_ID, "corr": correlation, "p_val": p_val, "z_score": z_score})

    # keep data
    region_connectivity_df = pd.DataFrame(region_df)
    per_region_data[region_pair[0] + "_" + region_pair[1]] = region_connectivity_df

    # save region_connectivity_df to csv
    region_connectivity_df.to_csv(f'{connectiviity_data_dir}/{region_pair[0]}_{region_pair[1]}_connectivity.csv', index=False)

    # # Plot the histogram of the Z-scores
    # plt.hist(region_connectivity_df.z_score, alpha=0.6, color='g')
    # plt.title("Histogram of Z-scores")
    # plt.show()

    # display(region_connectivity_df)


# create a data frame that has only the connectivity score (z-score) for each region pair (and the subject ID):\
connectivity_df = pd.DataFrame(columns=['subID'])
for key in per_region_data.keys():
    connectivity_score = per_region_data[key][['subID', 'z_score']]
    connectivity_score = connectivity_score.rename(columns={'z_score': key})
    connectivity_df = pd.merge(connectivity_df, connectivity_score, on='subID', how='outer')
# save the connectivity_df to csv
connectivity_df.to_csv(f'{connectiviity_data_dir}/all_connectivity_scores.csv', index=False)
display(connectivity_df)

Analyzing connectivity between L_SMA and L_post_putamen
Analyzing subject 101
>> Subject 101: correlation between L_SMA and L_post_putamen: r=0.6917043630972877, p=1.5139927139354282e-171, z-score=0.8512163079608521
Analyzing subject 102
>> Subject 102: correlation between L_SMA and L_post_putamen: r=0.6485072876694717, p=3.9040880115265387e-144, z-score=0.7727182526556606
Analyzing subject 103
>> Subject 103: correlation between L_SMA and L_post_putamen: r=0.6538507099040197, p=2.784511153356039e-147, z-score=0.7819957276054
Analyzing subject 104
>> Subject 104: correlation between L_SMA and L_post_putamen: r=0.6016496239526503, p=4.7583633557760423e-119, z-score=0.6957287161145137
Analyzing subject 105
>> Subject 105: correlation between L_SMA and L_post_putamen: r=0.4384188305117284, p=1.5511867074253952e-57, z-score=0.47027171644291077
Analyzing subject 106
>> Subject 106: correlation between L_SMA and L_post_putamen: r=0.5378816811456433, p=6.095912912642107e-91, z-score=0.6011701

,subID,L_SMA_L_post_putamen,R_SMA_R_post_putamen,L_premotor_L_post_putamen,R_premotor_R_post_putamen,L_anterior_caudate_vmpfc,R_anterior_caudate_vmpfc,L_vlpfc_L_post_putamen,R_vlpfc_R_post_putamen,L_vlpfc_L_anterior_putamen,R_vlpfc_R_anterior_putamen,L_vlpfc_L_anterior_caudate,R_vlpfc_R_anterior_caudate,R_frontopolar_R_anterior_caudate,R_frontopolar_R_post_putamen,R_post_putamen_L_post_putamen,R_anterior_caudate_L_anterior_caudate
0,101,0.851216,0.998780,0.758278,1.088896,0.160140,0.171255,0.186758,0.724928,0.074329,0.486687,0.268736,0.434427,0.509097,0.601175,0.837498,0.867571
1,102,0.772718,0.758162,0.784233,0.910795,0.497427,0.386723,0.470668,0.485592,0.519491,0.629122,0.529293,0.543271,0.492938,0.224427,0.608410,0.882937
2,103,0.781996,1.040435,0.862373,1.010847,0.816637,0.841093,0.652561,0.572853,0.461742,0.648554,0.563616,0.908927,0.787799,0.647073,0.975670,0.991395
3,104,0.695729,0.617592,0.520520,0.608107,0.232703,0.166782,0.179454,0.510091,0.147228,0.413492,0.357026,0.545893,0.495016,0.348266,1.046327,0.733302
4,105,0.470272,0.393517,0.472918,0.488532,0.421866,0.347436,0.374574,0.505057,0.310996,0.333324,0.451271,0.388881,0.339198,0.322174,0.877251,1.278251
5,106,0.601170,0.641163,0.634044,0.390154,0.388621,0.386643,0.612639,0.268618,0.487770,0.417695,0.437910,0.424911,0.619142,0.244757,0.610082,0.684291
6,107,0.837320,0.723076,0.920463,0.804596,0.262284,0.337655,0.245266,0.110878,0.215483,-0.051656,0.319452,0.164916,0.639618,0.357379,0.752838,0.979017
7,108,0.803382,0.986233,0.938801,1.088190,0.876531,1.093217,0.438439,0.905003,0.444636,0.862052,0.685624,0.996331,0.950917,0.696882,0.765659,1.389258
8,109,0.658739,0.711127,0.623994,0.872866,0.323784,0.330265,0.374150,0.621026,0.313722,0.540757,0.497184,0.532202,0.450668,0.499195,0.788206,0.786762
9,110,0.909104,0.886818,0.843693,1.112190,0.558206,0.681099,0.399795,0.813282,0.383113,0.647586,0.588102,0.919163,0.660548,0.634694,0.890801,1.104343


In [6]:
# show summary:
connectivity_df.describe()

,L_SMA_L_post_putamen,R_SMA_R_post_putamen,L_premotor_L_post_putamen,R_premotor_R_post_putamen,L_anterior_caudate_vmpfc,R_anterior_caudate_vmpfc,L_vlpfc_L_post_putamen,R_vlpfc_R_post_putamen,L_vlpfc_L_anterior_putamen,R_vlpfc_R_anterior_putamen,L_vlpfc_L_anterior_caudate,R_vlpfc_R_anterior_caudate,R_frontopolar_R_anterior_caudate,R_frontopolar_R_post_putamen,R_post_putamen_L_post_putamen,R_anterior_caudate_L_anterior_caudate
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.681448,0.729670,0.719305,0.851974,0.458582,0.499294,0.428985,0.613296,0.359040,0.545706,0.442980,0.580425,0.600919,0.474188,0.693495,0.872338
std,0.204702,0.245177,0.236099,0.256044,0.254991,0.261850,0.181817,0.249858,0.192625,0.230112,0.212223,0.246104,0.186129,0.218326,0.263329,0.284147
min,0.267304,0.163705,0.283654,0.231448,0.048606,0.166782,0.175650,0.073992,-0.071605,-0.051656,0.045739,0.046334,0.252086,0.034001,0.097700,0.119509
25%,0.575622,0.606687,0.539405,0.662364,0.284843,0.321822,0.307544,0.444906,0.245013,0.383678,0.298611,0.422347,0.469956,0.314722,0.552217,0.691526
50%,0.678138,0.718625,0.695094,0.849450,0.398816,0.449509,0.374362,0.575234,0.318149,0.532846,0.408859,0.546757,0.597825,0.497280,0.721875,0.852485
75%,0.830492,0.910371,0.885628,1.039561,0.553041,0.610876,0.551606,0.807151,0.456366,0.651587,0.528921,0.683808,0.710629,0.621677,0.871255,1.023201
max,1.135984,1.236386,1.290920,1.440720,1.477396,1.345420,0.838888,1.318569,0.783995,1.188736,1.063276,1.251956,1.102976,1.146660,1.177976,1.588605
